In [1]:
import sys
import os
import time as t

import numpy as np
import pandas as pd

In [2]:
# raw data가 들어있는 폴더명
# .(콤마)는 현위치를 의미. 현 위치의 하위 폴더 ./폴더명

path_false = "./data/20190709/"
path_true = "./data/20190709/"

In [3]:
# 추출할 feature

cc = [
    "subject", "age", "group",
    "R_HIP_Flex_ANG", "L_HIP_Flex_ANG",
    "R_KNEE_Flex_ANG", "L_KNEE_Flex_ANG",
    "R_ANK_Flex_ANG", "L_ANK_Flex_ANG",
    "R_Pelvis_Lat_Tilt", "L_Pelvis_Lat_Tilt",
    "R_HIP_Rot_ANG", "L_HIP_Rot_ANG",
    "R_Foot_Orientation", "L_Foot_Orientation"
]

In [4]:
# xlsx 파일에서 data와 age를 읽어오는 함수

def read_files(path, file):
    data = pd.read_excel(
        os.path.join(path, file),
        sheet_name="Average",
        header=1,
        skiprows=27,)
    try:
        age = pd.read_excel(
            os.path.join(path, file),
            skipfooter=17,
            header=1,
        )["Unnamed: 16"][6]
    except Exception as e:
        print(file)
        age = np.nan
    return data, age

In [5]:
# 파일 저장용 함수

def save_data(result, group: str = "all"):
    now = t.localtime()
    folder_name = "result"
    try:
        if not(os.path.isdir(folder_name)):
            os.makedirs(os.path.join(folder_name))
    except OSError as e:
        if e.errno != errno.EEXIST:
            print("Failed to create directory!!!!!")
            raise

    s = "%04d%02d%02d_%02d%02d" % (now.tm_year, now.tm_mon,
                                   now.tm_mday, now.tm_hour, now.tm_min)

    # 파일 저장
    result.to_csv("./{}/gait_{}_{}.csv".format(folder_name, s, group),
                  encoding='cp949', index=False)

In [6]:
# 파일 읽는 순서 상관 있는지? 

# key=lambda x: int(x.split(".")[0]))
files_false = sorted(os.listdir(path_false),)
files_false = [f for f in files_false if ".xl" in f] # 엑셀 파일만 가능하도록
files_false

['001.xls', '002.xls', '003.xls']

In [8]:
%%time

# 일반적인 table로 생성 후, 가로로 붙이는 작업을 한다.

subject = 0
result = pd.DataFrame()

for file in files_false:
    print(file)
    data, age = read_files(path_false, file)
    group = 0
    data["subject"] = subject
    data["age"] = age
    data["group"] = group
    data = data[cc]

    add_col = []
    if result.empty:
        feature_cc = cc[3:]
        for i in range(len(feature_cc)):
            for j in range(1,101):
                add_col.append(feature_cc[i]+'_{:03}'.format(j))
        result_col = cc[:3]+add_col
        result = pd.DataFrame(columns=result_col)
    
    # insert data to table
    longi_data = []
    #print(data[data.columns[3]])
    
    for i in range(3,len(cc)):
        temp = [j for j in data[data.columns[i]].values.tolist()]
        longi_data+=temp
    
    temp = pd.DataFrame([[subject,  age, group, ] + longi_data], columns=result_col)
    result = result.append(temp, ignore_index=True)

    subject += 1

001.xls
002.xls
003.xls
Wall time: 22.5 s


In [9]:
# save group = false
save_data(result=result, group="false")